In [1]:
!pip install torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00


In [2]:
import torch
from argparse import ArgumentParser
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2LMHeadModel, AutoModel, AutoTokenizer, AutoModelForCausalLM

parser = ArgumentParser()
parser.add_argument("-m", "--model-path", type=str, required=True)
parser.add_argument("-o", "--output-path", type=str, required=True)
parser.add_argument("-b", "--num-beams", type=int, default=5)

'''
언어 생성 모델에서 빔서치(Beam Search)를 사용할 때 빔의 개수를 지정하는 옵션입니다.
빔서치는 생성 모델이 다음 단어를 예측할 때 사용되는 방법 중 하나로,
여러 개의 후보 단어를 유지하고 확률적으로 가장 적합한 단어를 선택합니다.
'''


def generate_question(context, num_beams=5):

    # 모델 경로 위치 잡기
    model = GPT2LMHeadModel.from_pretrained("sangdal/ChatBot")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # tokenizer폴더 경로위치 잡기
    tokenizer = SentencePieceBPETokenizer.from_file(
        vocab_filename="/content/drive/MyDrive/프로젝트(web)/tokenizer/vocab.json", merges_filename="/content/drive/MyDrive/프로젝트(web)/tokenizer/merges.txt", add_prefix_space=False
    )

    example = {"context": context, "question": "", "answer": ""}
    # example = {"context": context, "question": "당신의 장점은? ", "answer": "착함"}

    inputs = tokenizer.encode(example["context"])
    input_ids = torch.tensor(inputs.ids, dtype=torch.long).unsqueeze(0).to(device)  # Specify dtype as torch.long


    model = model.to(device)
    model.eval()

    generated_results = []

    origin_seq_len = input_ids.size(-1)

    decoded_sequences = model.generate(
        input_ids=input_ids,
        max_length=origin_seq_len + 100, # 질문의 최대길이
        min_length=origin_seq_len + 5, # 질문의 최소길이
        pad_token_id=0,
        bos_token_id=1,
        eos_token_id=2,
        num_beams=num_beams,
        repetition_penalty=1.3,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
    )

    for decoded_tokens in decoded_sequences.tolist():
        decoded_question_text = tokenizer.decode(decoded_tokens[origin_seq_len:])
        decoded_question_text = decoded_question_text.split("</s>")[0].replace("<s>", "")
        decoded_question_text = decoded_question_text.split("질문:")[-1]
        generated_results.append(decoded_question_text)

    return generated_results



In [3]:

# if __name__ == "__main__":
context = input("문맥을 입력하세요: ")
# num_beams = int(input("num_beams를 입력하세요 (기본값: 5): ") or 5)

generated_question = generate_question(context)

print(f"생성된 질문: {generated_question}")


문맥을 입력하세요: 저는 한국석유공사에서 기계 분야의 기계 설계 파트를 가장 잘 수행할 수 있습니다. 저는 고등학교 생활 당시 선택과목으로 기계 설계 요소를 선택하여 수업을 했던 경험이 있습니다. 처음 기계 설계 요소 과목을 선택 했을때는 남들보다 잘 하지 못하여 흥미를 느끼지 못하였지만 점점하다 보니 흥미를 가지게 되었으며 나중에는 다른 친구들보다 잘하고 싶다는 생각을 가지게 되어 수업 시간 이외의 시간에도 기계 설계 요소 도면을 찾아보며 매일 한 시간씩 그리게 되었습니다. 이러한 노력들을 통해 저의 설계 실력이 점점 늘어난다는 것을 느끼게 되었으며 이후 수업 시간에 선생님이 주시는 도면들도 가장 먼저 만들며 틀리지 않아 친구들에게 가르쳐 주기도 하였습니다. 이후 저는 최종적으로 설계 과목 1등을 하여 교과 우수상을 받게 되었습니다. 이러한 경험들을 바탕으로 앞으로도 기계 설계 공부를 소홀히 하지 않으며 관련 자격증을 취득하여 능력을 더욱 올릴 것이며 일반 기계기사 자격증을 취득하며 기계 분야를 더욱 공부할 것입니다. 또한 한국석유공사에서도 저의 분야가 아닌 전기, 건설 등의 직무에서도 모르는 부분이 있다면 직원 동료분들에게 물어보며 도움이 되기 위해 노력할 것입니다.


생성된 질문: ['한국석유공사에서의 기계 설계 업무에서 어떤 어려움이 있었나요?']
